In [17]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/llm-7-prompt-training-dataset/train_essays_RDizzl3_seven_v2.csv
/kaggle/input/llm-7-prompt-training-dataset/train_essays_7_prompts_v2.csv
/kaggle/input/llm-7-prompt-training-dataset/train_essays_7_prompts.csv
/kaggle/input/llm-7-prompt-training-dataset/train_essays_RDizzl3_seven_v1.csv


In [18]:
import pandas as pd
df1=pd.read_csv('/kaggle/input/llm-7-prompt-training-dataset/train_essays_7_prompts.csv')
df2=pd.read_csv('/kaggle/input/llm-7-prompt-training-dataset/train_essays_7_prompts_v2.csv')
df3=pd.read_csv('/kaggle/input/llm-7-prompt-training-dataset/train_essays_RDizzl3_seven_v1.csv')
df4=pd.read_csv('/kaggle/input/llm-7-prompt-training-dataset/train_essays_RDizzl3_seven_v2.csv')


In [19]:

print(f'df1 shape:',df1.shape)
print(f'df2 shape:',df2.shape)
print(f'df3 shape:',df3.shape)
print(f'df4 shape:',df4.shape)

df1 shape: (14877, 2)
df2 shape: (15350, 2)
df3 shape: (15871, 2)
df4 shape: (17251, 2)


In [20]:
print(df1.columns)
print(df2.columns)
print(df3.columns)
print(df4.columns)


Index(['text', 'label'], dtype='object')
Index(['text', 'label'], dtype='object')
Index(['text', 'label'], dtype='object')
Index(['text', 'label'], dtype='object')


In [21]:
df_all = pd.concat([df1, df2, df3, df4], ignore_index=True)
print(df_all.shape)
df_all.head()


(63349, 2)


,text,label
0,Cars. Cars have been around since they became ...,0
1,Transportation is a large necessity in most co...,0
2,"""America's love affair with it's vehicles seem...",0
3,How often do you ride in a car? Do you drive a...,0
4,Cars are a wonderful thing. They are perhaps o...,0


In [22]:
print(f'nulls in dataset',df_all.isnull().sum())

nulls in dataset text     0
label    0
dtype: int64


In [23]:
from sklearn.model_selection import train_test_split

df_all = df_all.sample(frac=1, random_state=42).reset_index(drop=True)
train_df, val_df = train_test_split(df_all, test_size=0.1, random_state=42)

In [24]:
from torch.utils.data import Dataset, DataLoader
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = 'huawei-noah/TinyBERT_General_4L_312D'
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Define your custom dataset
class MyDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=64):
        self.encodings = tokenizer(
            texts,
            padding='max_length',      # pad to max_length
            truncation=True,
            max_length=max_length,     # shorter input to save memory
            return_tensors='pt'        # directly return PyTorch tensors
        )
        self.labels = torch.tensor(labels)

    def __getitem__(self, idx):
        return {
            'input_ids': self.encodings['input_ids'][idx],
            'attention_mask': self.encodings['attention_mask'][idx],
            'labels': self.labels[idx]
        }

    def __len__(self):
        return len(self.labels)


#####################################################

train_texts = train_df['text'].tolist()
train_labels = train_df['label'].tolist()

# Prepare validation data
val_texts = val_df['text'].tolist()
val_labels = val_df['label'].tolist()

# Create Dataset instances
train_dataset = MyDataset(train_texts, train_labels, tokenizer)
val_dataset = MyDataset(val_texts, val_labels, tokenizer)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

# Create dataset and dataloader
dataset = MyDataset(texts, labels, tokenizer)
loader = DataLoader(dataset, batch_size=16, shuffle=True)


In [25]:
from transformers import AutoModelForSequenceClassification
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f"Using device: {device}")


Using device: cuda


In [26]:
from transformers import AutoModelForSequenceClassification

model_name = 'huawei-noah/TinyBERT_General_4L_312D'  

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
model.to(device)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at huawei-noah/TinyBERT_General_4L_312D and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 312, padding_idx=0)
      (position_embeddings): Embedding(512, 312)
      (token_type_embeddings): Embedding(2, 312)
      (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-3): 4 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=312, out_features=312, bias=True)
              (key): Linear(in_features=312, out_features=312, bias=True)
              (value): Linear(in_features=312, out_features=312, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=312, out_features=312, bias=True)
              (LayerNorm): LayerNorm((312,), eps=1e-1

In [27]:
from torch.optim import AdamW
optimizer=AdamW(model.parameters(),lr=5e-5,eps=1e-8,weight_decay=0.01)

In [28]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU found")


True
Tesla T4


In [29]:
from transformers import get_scheduler
from sklearn.metrics import classification_report
from tqdm import tqdm
import torch

num_epochs = 2

# Scheduler setup
num_training_steps = num_epochs * len(train_loader)
scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)

for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    print(f"\nEpoch {epoch + 1}/{num_epochs}")
    for batch in tqdm(train_loader):
        batch = {k: v.to(device) for k, v in batch.items()}

        outputs = model(**batch)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

    avg_loss = total_loss / len(train_loader)
    print(f"Average training loss: {avg_loss:.4f}")

    # Evaluation on validation set
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch in val_loader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(input_ids=batch['input_ids'], attention_mask=batch['attention_mask'])
            logits = outputs.logits
            preds = torch.argmax(logits, dim=-1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(batch['labels'].cpu().numpy())

    print("Validation classification report:")
    print(classification_report(all_labels, all_preds, digits=4))



Epoch 1/2


100%|██████████| 3564/3564 [00:59<00:00, 59.88it/s]


Average training loss: 0.0490
Validation classification report:
              precision    recall  f1-score   support

           0     0.9980    0.9980    0.9980      5579
           1     0.9854    0.9854    0.9854       756

    accuracy                         0.9965      6335
   macro avg     0.9917    0.9917    0.9917      6335
weighted avg     0.9965    0.9965    0.9965      6335


Epoch 2/2


100%|██████████| 3564/3564 [01:00<00:00, 58.43it/s]


Average training loss: 0.0049
Validation classification report:
              precision    recall  f1-score   support

           0     0.9995    0.9995    0.9995      5579
           1     0.9960    0.9960    0.9960       756

    accuracy                         0.9991      6335
   macro avg     0.9977    0.9977    0.9977      6335
weighted avg     0.9991    0.9991    0.9991      6335



In [30]:
import torch
from torch.nn import CrossEntropyLoss

from collections import Counter
label_counts = Counter(labels)  
total_count = sum(label_counts.values())

class_weights = [total_count / label_counts[i] for i in range(len(label_counts))]
class_weights = torch.tensor(class_weights).to(device)

loss_fn = CrossEntropyLoss(weight=class_weights)

outputs = model(input_ids=batch['input_ids'], attention_mask=batch['attention_mask'])
logits = outputs.logits

loss = loss_fn(logits, batch['labels'])


In [33]:
from sklearn.metrics import classification_report
import numpy as np

model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in val_loader:  
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(input_ids=batch['input_ids'], attention_mask=batch['attention_mask'])
        logits = outputs.logits
        preds = torch.argmax(logits, dim=-1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(batch['labels'].cpu().numpy())

print(classification_report(all_labels, all_preds, digits=4))


              precision    recall  f1-score   support

           0     0.9995    0.9995    0.9995      5579
           1     0.9960    0.9960    0.9960       756

    accuracy                         0.9991      6335
   macro avg     0.9977    0.9977    0.9977      6335
weighted avg     0.9991    0.9991    0.9991      6335



In [35]:
model.save_pretrained("finetuned_distilbert")
tokenizer.save_pretrained("finetuned_distilbert")


('finetuned_distilbert/tokenizer_config.json',
 'finetuned_distilbert/special_tokens_map.json',
 'finetuned_distilbert/vocab.txt',
 'finetuned_distilbert/added_tokens.json',
 'finetuned_distilbert/tokenizer.json')

In [39]:
#will check accuracy for unseen data 

df_unseen = df_all.iloc[int(0.9 * len(df_all)):].copy()


texts_unseen = df_unseen['text'].tolist()
labels_unseen = df_unseen['label'].tolist()  
encodings_unseen = tokenizer(texts_unseen, padding=True, truncation=True, max_length=512)

class MyDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        return {
            'input_ids': torch.tensor(self.encodings['input_ids'][idx]),
            'attention_mask': torch.tensor(self.encodings['attention_mask'][idx]),
            'labels': torch.tensor(self.labels[idx])
        }

    def __len__(self):
        return len(self.labels)

unseen_dataset = MyDataset(encodings_unseen, labels_unseen)
unseen_loader = DataLoader(unseen_dataset, batch_size=16)


In [40]:
from sklearn.metrics import classification_report

model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in unseen_loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(input_ids=batch['input_ids'], attention_mask=batch['attention_mask'])
        preds = torch.argmax(outputs.logits, dim=-1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(batch['labels'].cpu().numpy())

print(classification_report(all_labels, all_preds, digits=4))


              precision    recall  f1-score   support

           0     0.9743    0.9773    0.9758      5633
           1     0.8131    0.7934    0.8032       702

    accuracy                         0.9569      6335
   macro avg     0.8937    0.8854    0.8895      6335
weighted avg     0.9565    0.9569    0.9567      6335



In [48]:
model.save_pretrained("my_model")
tokenizer.save_pretrained("my_model")


('my_model/tokenizer_config.json',
 'my_model/special_tokens_map.json',
 'my_model/vocab.txt',
 'my_model/added_tokens.json',
 'my_model/tokenizer.json')